In [2]:
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import cross_validate

In [3]:
data = Dataset.load_builtin('ml-100k',prompt = False)
data.raw_ratings[:10]

Trying to download dataset from http://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to C:\Users\wooyong/.surprise_data/ml-100k


[('196', '242', 3.0, '881250949'),
 ('186', '302', 3.0, '891717742'),
 ('22', '377', 1.0, '878887116'),
 ('244', '51', 2.0, '880606923'),
 ('166', '346', 1.0, '886397596'),
 ('298', '474', 4.0, '884182806'),
 ('115', '265', 2.0, '881171488'),
 ('253', '465', 5.0, '891628467'),
 ('305', '451', 3.0, '886324817'),
 ('6', '86', 3.0, '883603013')]

In [4]:
model = SVD()

In [6]:
cross_validate(model, data, measures = ['rmse','mae'], cv = 5, verbose = True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9306  0.9366  0.9407  0.9320  0.9485  0.9377  0.0065  
MAE (testset)     0.7320  0.7387  0.7413  0.7360  0.7470  0.7390  0.0050  
Fit time          4.32    4.30    4.35    4.43    4.35    4.35    0.04    
Test time         0.18    0.14    0.19    0.18    0.19    0.18    0.02    


{'test_rmse': array([0.93056905, 0.93662951, 0.9407038 , 0.93201397, 0.94854033]),
 'test_mae': array([0.73202427, 0.73867883, 0.74132235, 0.73598894, 0.74701216]),
 'fit_time': (4.31846809387207,
  4.3023881912231445,
  4.349832773208618,
  4.427505254745483,
  4.351675987243652),
 'test_time': (0.18364405632019043,
  0.14150214195251465,
  0.19225215911865234,
  0.18163752555847168,
  0.19004273414611816)}

Content based filtering

In [7]:
import numpy as np
from surprise import Dataset

In [8]:
data = Dataset.load_builtin('ml-100k', prompt = False)
raw_data = np.array(data.raw_ratings,dtype = int)

In [9]:
raw_data[:,0] -= 1
raw_data[:,1] -= 1

In [10]:
n_users = np.max(raw_data[:, 0])
n_movies = np.max(raw_data[:,1])
shape = (n_users + 1, n_movies + 1)
shape

(943, 1682)

In [16]:
adj_matrix = np.ndarray(shape, dtype = int)
for user_id, movie_id, rating, time in raw_data:
    adj_matrix[user_id][movie_id] = 1.
adj_matrix

array([[1, 1, 1, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0]])

In [20]:
my_id, my_vector = 0, adj_matrix[0]
best_match, best_match_id, best_match_vector = -1, -1, []

for user_id, user_vector, in enumerate(adj_matrix):
    if my_id != user_id:
        similarity = np.dot(my_vector, user_vector)
        if similarity > best_match:
            best_match = similarity
            best_match_id = user_id
            best_match_vector = user_vector
            
print('BestMatch: {}, best Match Id: {}'.format(best_match,best_match_id))

BestMatch: 183, best Match Id: 275


In [25]:
recommend_list = []
for i, log in enumerate(zip(my_vector, best_match_vector)):
    log1, log2 = log
    if log1 < 1. and log2 > 0.:
       recommend_list.append(i)
print(recommend_list)

[272, 273, 275, 280, 281, 283, 287, 288, 289, 290, 292, 293, 297, 299, 300, 301, 302, 306, 312, 314, 315, 316, 317, 321, 322, 323, 324, 327, 330, 331, 332, 333, 339, 342, 345, 346, 353, 354, 355, 356, 357, 363, 364, 365, 366, 372, 374, 378, 379, 381, 382, 383, 384, 385, 386, 387, 390, 391, 392, 394, 395, 396, 398, 400, 401, 402, 403, 404, 405, 406, 407, 408, 409, 410, 412, 414, 416, 417, 418, 419, 420, 422, 424, 425, 426, 427, 428, 430, 431, 432, 435, 442, 446, 447, 448, 449, 450, 451, 452, 454, 455, 457, 460, 461, 462, 468, 469, 470, 471, 472, 473, 474, 478, 495, 500, 507, 517, 522, 525, 530, 539, 540, 543, 545, 546, 548, 549, 550, 551, 553, 557, 558, 560, 561, 562, 563, 565, 566, 567, 568, 570, 571, 574, 575, 576, 577, 580, 581, 582, 585, 587, 589, 590, 594, 596, 602, 623, 626, 627, 630, 633, 635, 639, 646, 648, 651, 652, 654, 657, 664, 668, 671, 677, 678, 681, 683, 684, 685, 690, 691, 692, 695, 696, 708, 709, 714, 718, 719, 720, 724, 726, 727, 731, 733, 734, 736, 738, 741, 742, 745,

In [ ]:
my_id, my_vector = 0, adj_matrix[0]
best_match, best_match_id, best_match_vector = 9999, -1, []

for user_id, user_vector, in enumerate(adj_matrix):
    if my_id != user_id:
        similarity = np.dot(my_vector, user_vector)
        if similarity > best_match:
            best_match = similarity
            best_match_id = user_id
            best_match_vector = user_vector
            
print('BestMatch: {}, best Match Id: {}'.format(best_match,best_match_id))